In [1]:
from bilibili_api import bangumi, sync
from bilibili_api import settings
import numpy as np 
import nest_asyncio
 
nest_asyncio.apply()

### Invoking Bilibili-api & Loading all the short comments as raw dataset

In [2]:
async def main(media_id):
    b = bangumi.Bangumi(media_id)
    next = None
    step = 0
    cmts = []
    cmt_X = np.array([])
    cmt_y = np.array([])
    while next != 0 and step<2000:
        if step%10==0 :
            print("Crawling at", step//10)
        step = step + 1
        cm = await b.get_short_comment_list(next=next)
        cmts.extend(cm['list'])
        next = cm['next']

    for cmt in cmts:
        cmt_X = np.append(cmt_X, cmt['content'])
        cmt_y = np.append(cmt_y, cmt['score'])
    
    return cmt_X, cmt_y

In [3]:
X, y = sync(main(4315402))

Crawling at 0
Crawling at 1
Crawling at 2
Crawling at 3
Crawling at 4
Crawling at 5
Crawling at 6
Crawling at 7
Crawling at 8
Crawling at 9
Crawling at 10
Crawling at 11
Crawling at 12
Crawling at 13
Crawling at 14
Crawling at 15
Crawling at 16
Crawling at 17
Crawling at 18
Crawling at 19
Crawling at 20
Crawling at 21
Crawling at 22
Crawling at 23
Crawling at 24
Crawling at 25
Crawling at 26
Crawling at 27
Crawling at 28
Crawling at 29
Crawling at 30
Crawling at 31
Crawling at 32
Crawling at 33
Crawling at 34
Crawling at 35
Crawling at 36
Crawling at 37
Crawling at 38
Crawling at 39
Crawling at 40
Crawling at 41
Crawling at 42
Crawling at 43
Crawling at 44
Crawling at 45
Crawling at 46
Crawling at 47
Crawling at 48
Crawling at 49
Crawling at 50
Crawling at 51
Crawling at 52
Crawling at 53
Crawling at 54
Crawling at 55
Crawling at 56
Crawling at 57
Crawling at 58
Crawling at 59
Crawling at 60
Crawling at 61
Crawling at 62


In [4]:
np.sum(y<6)

3152

In [5]:
np.sum(y)

84764.0

In [6]:
X[0]

'长评已更新'

###  Datasets Format Transformation through the Pandas Library

In [7]:
from pandas import DataFrame

data = { 'label': (y>=6), 'score': y, 'content': X }
df = DataFrame(data)
df.to_excel('../corpus/BiliAnimeReviews.xlsx')
# 全部导入 xlsx 文档中


In [8]:
# Consider positive texts as comments whose rating is greater than 6
X_pos = X[y>6]
X_pos

array(['赞！期待！', '能出来就是胜利',
       '第三集刚看完，确实觉得时长不够，又要等一周了。几年前看完小说，小说细节有点模糊，三集下来感觉还不错的，几个场景印象很深，三体游戏，古筝行动 ，粒子对撞实验，剧情和画面都挺流畅，会员不亏！',
       ..., '这集和前几集串一起的话还是不错的', '它还原了很多书上的内容，不喜欢看书的，可以来看这个', '反正符合我的预期！'],
      dtype='<U100')

In [9]:
# Consider negative texts as comments whose rating is the opposent
X_neg = X[y<=6]
X_neg

array(['长评已更新', '有提升空间', '目前看来在预期之内，后续观望', ...,
       '预测分数可能在8分以上，所以我打了8分 // 瓶皇都不装了，那我也没必要客气了', '三星观望一下',
       '如果不叫三体还好，但这就不是三体'], dtype='<U100')

In [10]:
# save corpus in txt documents
np.savetxt('../corpus/pos.txt', X_pos, encoding='utf-8', fmt='%s')
np.savetxt('../corpus/neg.txt', X_neg, encoding='utf-8', fmt='%s')

### Design two extreme categories for pos & neg

In [11]:
X_pos_extreme = X[y>=8]
X_neg_extreme = X[y<=2]

In [13]:
np.savetxt('../corpus/pos_ex.txt', X_pos_extreme, encoding='utf-8', fmt='%s')
np.savetxt('../corpus/neg_ex.txt', X_neg_extreme, encoding='utf-8', fmt='%s')